In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('bq_data.csv')
df = df.loc[df['receiver'].notnull()]
df = df.loc[df['category'].notnull()]
display(df)
print(df.info())

,date,commit_date,receiver,amount,category,commit_file_id
1,2023-04-20,2023-05-03,H M,-39.99,HOUSEHOLD-ITEMS,POP
2,2023-04-24,2023-05-03,STADIUM,-66.48,HOBBIES,POP
3,2023-04-16,2023-05-03,KLARNA AB,-43.94,HOUSEHOLD-ITEMS,POP
4,2023-04-12,2023-05-03,KLARNA AB,-221.30,UNCATEGORIZED,POP
5,2023-04-03,2023-05-03,CHF*PNTpay,-5.00,ENTERTAINMENT,POP
...,...,...,...,...,...,...
1188,2023-03-06,2023-04-04,SEPPÄLÄ SEIJA TUULIKKI,25.00,OTHER-INCOME,POP
1189,2023-03-21,2023-04-04,IKEA VANTAA IF CUSTOMER R,-9.68,FOOD,POP
1190,2023-03-20,2023-04-04,K-CITYMARKET ESPOO LEPPAV,-75.68,FOOD,POP
1191,2023-03-10,2023-04-04,MOBILEPAY KATARIINA SEPPÄLÄ,100.00,OTHER-INCOME,POP


<class 'pandas.core.frame.DataFrame'>
Index: 1130 entries, 1 to 1192
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            1130 non-null   object 
 1   commit_date     1130 non-null   object 
 2   receiver        1130 non-null   object 
 3   amount          1130 non-null   float64
 4   category        1130 non-null   object 
 5   commit_file_id  1130 non-null   object 
dtypes: float64(1), object(5)
memory usage: 61.8+ KB
None


In [17]:
test = df[['amount', 'receiver']].to_numpy()
for value in test[:, 1]:
    assert isinstance(value, float), "First"

AssertionError: First

In [3]:
from time import monotonic_ns
def my_timing(func):
    def wrapper(*args, **kwargs):
        t0 = monotonic_ns()
        values = func(*args, **kwargs)
        t1 = monotonic_ns()
        print(f'Function: {func.__name__}')
        print(f'With args: [{args}, {kwargs}]')
        print(f'Took {(t1 - t0)/1_000_000_000:.4f}s to run.\n')
        return values
    return wrapper

In [4]:
def encode_variables(data: np.ndarray) -> dict:
    return {i: label for i, label in enumerate(np.unique(data))}

In [5]:
def learn_y(y):
    decode = {i: label for i, label in enumerate(np.unique(y))}
    encode = {label: i for i, label in enumerate(np.unique(y))}
    return encode, decode

@my_timing
def transform_y(y, encode):
    return [encode[label] for label in y]


y = df['category'].to_list()
encode, decode = learn_y(y)
y = transform_y(y, encode=encode)

priors = {label: y.count(label) for label in y}

priors = {decode[label]: count for label, count in priors.items()}

display(priors)

Function: transform_y
With args: [(['HOUSEHOLD-ITEMS', 'HOBBIES', 'HOUSEHOLD-ITEMS', 'UNCATEGORIZED', 'ENTERTAINMENT', 'COMMUTING', 'COMMUTING', 'COMMUTING', 'COMMUTING', 'COMMUTING', 'SALARY', 'TECHNOLOGY', 'COMMUTING', 'LIVING', 'TECHNOLOGY', 'HOUSEHOLD-ITEMS', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'HOBBIES', 'HOBBIES', 'INVESTING', 'CLOTHING', 'OTHER-INCOME', 'HOBBIES', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'HOBBIES', 'ENTERTAINMENT', 'TECHNOLOGY', 'FOOD', 'LIVING', 'FOOD', 'UNCATEGORIZED', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'OTHER-INCOME', 'FOOD', 'OTHER-INCOME', 'CLOTHING', 'HEALTH', 'FOOD', 'OTHER-INCOME', 'OTHER-INCOME', 'OTHER-INCOME', 'CLOTHING', 'LIVING', 'ENTERTAINMENT', 'TECHNOLOGY', 'TECHNOLOGY', 'COMMUTING', 'COMMUTING', 'COMMUTING', 'COMMUTING', 'COMMUTING', 'COMMUTING', 'SALARY', 'TECHNOLOGY', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'COMMUTING', 'COMMUTING', 'COMMUTING', 'LIVING', 'TECHNOLOGY', 'OTHER-INCO

{'HOUSEHOLD-ITEMS': 64,
 'HOBBIES': 56,
 'UNCATEGORIZED': 34,
 'ENTERTAINMENT': 87,
 'COMMUTING': 100,
 'SALARY': 23,
 'TECHNOLOGY': 105,
 'LIVING': 51,
 'FOOD': 435,
 'INVESTING': 18,
 'CLOTHING': 25,
 'OTHER-INCOME': 98,
 'HEALTH': 34}

In [7]:
@my_timing
def learn_vocabulary(X):
    nested_word_list = [re.split('\.|;|,|-|_|\*|\s+', str(row).lower()) for row in X]
    word_list = set(value for nested in nested_word_list for value in nested)
    word_dict = {word: i for i, word in enumerate(word_list)}
    return word_dict


X = df['receiver'].to_numpy()
corpus = learn_vocabulary(X)
display(corpus)

Function: learn_vocabulary
With args: [(array(['H M', 'STADIUM', 'KLARNA AB', ..., 'K-CITYMARKET ESPOO LEPPAV',
       'MOBILEPAY KATARIINA SEPPÄLÄ',
       'MOBILEPAY SANNA MAARIT MARJA HEIKKIL'], dtype=object),), {}]
Took 0.0054s to run.



{'': 0,
 'osuuspankki': 1,
 'kotipizza': 2,
 'swedavia': 3,
 'kalalahti': 4,
 'kela': 5,
 'aal': 6,
 'ravintola': 7,
 'kobenhavn': 8,
 'maaret': 9,
 'adomex': 10,
 'corner': 11,
 'eco': 12,
 'helsi': 13,
 'luckiefun': 14,
 'oskar': 15,
 'colu': 16,
 'enterpri': 17,
 'casino': 18,
 'at': 19,
 'jumbo': 20,
 'apteekki': 21,
 'cust': 22,
 'amos': 23,
 'verkkokauppa': 24,
 'ya': 25,
 'service': 26,
 'systems': 27,
 '50': 28,
 'karisman': 29,
 'vuosa': 30,
 'centaurus': 31,
 'bikecompone': 32,
 'kiviston': 33,
 'dna': 34,
 'visa': 35,
 'market': 36,
 'filial': 37,
 'kaari': 38,
 '257': 39,
 'coop': 40,
 'leppavaara': 41,
 'paytrail': 42,
 'itunes': 43,
 'vaajakoski': 44,
 'delisis': 45,
 'teknologforeningen': 46,
 'kaunispaan': 47,
 'h': 48,
 'mai': 49,
 'etsyireland': 50,
 'bestseller': 51,
 'oy': 52,
 'kiinteistöhuolto': 53,
 'lensway': 54,
 'safiirikuj': 55,
 'hiekkamies': 56,
 'svea': 57,
 'alko': 58,
 'vantaa': 59,
 'dressmann': 60,
 'telia': 61,
 'mroom': 62,
 '&': 63,
 'softw': 64,
 '

In [27]:
@my_timing
def in_test(val, data):
    if val in data: 
        return True
    else: 
        return False

in_test(val='ei oo', data=corpus)

Function: in_test
With args: [(), {'val': 'ei oo', 'data': {'': 0, 'marja': 1, 'tripla': 2, 'l': 3, 'welho': 4, 'konnichiwa': 5, 'delisis': 6, 'espoon': 7, '7802': 8, 'ry': 9, 'caruna': 10, 'ky': 11, 'jarkko': 12, 'pop': 13, 'seinaejoki': 14, 'yhtymä': 15, 'mroom': 16, 'keychron': 17, 'casino': 18, 'c': 19, 'arlanda': 20, 'sell': 21, 'jj': 22, 'susannan': 23, 'amos': 24, 'jysk': 25, 'saati': 26, 'buffa': 27, 'estonia': 28, 'kebab': 29, 'vuotie': 30, 'center': 31, 'veke': 32, 'kira': 33, '207': 34, 'teknologforeningen': 35, 'katjan': 36, 'sport': 37, 'finland': 38, 'paivan': 39, 'ravinto': 40, 'b': 41, 'fiskars': 42, 'columbus': 43, 'colum': 44, 'teräsvalli': 45, 'payments': 46, 'avecra': 47, 'hels': 48, 'kaari': 49, 'checkout': 50, 'saiturinporssi': 51, 'finnball': 52, 'elias': 53, 'tower': 54, 'kotikontu': 55, 'spr': 56, 'services': 57, '65975': 58, 'laitaan': 59, 'amzn': 60, 'joona': 61, '044': 62, 'polyteknikkojen': 63, 'kaivoksela': 64, 'stockmann': 65, 'subway': 66, 'eco': 67, 'os

False

In [46]:
a = {'a': 1, 'b': 2, 'c': 3}

a[['a', 'c']]

TypeError: unhashable type: 'list'